# Project: Facial Emotion Recognition Predictor

## Dependencies

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sklearn

import tensorflow as tf
from tensorflow import keras

import pickle

sns.set_style("whitegrid")
sns.set_palette('muted')

In [16]:
for package in [pd, np, mpl, sns, sklearn, pickle, tf]:
    if package == pickle: print(f'{package.__name__:<12}: {package.format_version}')
    else: print(f'{package.__name__:<12}: {package.__version__}')

pandas      : 2.1.1
numpy       : 1.26.2
matplotlib  : 3.8.0
seaborn     : 0.12.2
sklearn     : 1.3.0
pickle      : 4.0
tensorflow  : 2.10.0


## Data Loading

The Kaggle dataset for this project can be downloaded from: 
```
https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data
```

Details of the dataset, including description of columns and associated datatypes, can be found [here](../data/README.md).

* Load all the data
* Separate into image arrays and labels
* Separate into training, validation and test sets

In [73]:
data = pd.read_csv('../data/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')

data.columns = data.columns.str.strip()

In [74]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

In [75]:
train_image_array, train_image_label = prepare_data(data[data['Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data['Usage']=='PublicTest'])
test_image_array, test_image_label = prepare_data(data[data['Usage']=='PrivateTest'])